<a href="https://colab.research.google.com/github/student64-ahmadi/odw-2020/blob/master/l'hypoth%C3%A8se_multidimensionnelle_ypnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas

In [ ]:
pip install numpy

In [ ]:
pip install matplotlib

In [ ]:
!pip install tensorflow
!pip install keras

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split


In [ ]:
!pip install gwpy

In [ ]:
! pip install -q 'gwpy==3.0.8'

In [ ]:
from gwosc.datasets import event_gps
gps = event_gps('GW170817')
print(gps)

1187008882.4


In [ ]:
segment = (int(gps)-5, int(gps)+5)
print(segment)

(1187008877, 1187008887)


In [ ]:
from gwpy.timeseries import TimeSeries
hdata = TimeSeries.fetch_open_data('H1', *segment, verbose=True)
print(hdata)

Fetched 1 URLs from gwosc.org for [1187008877 .. 1187008887))
Reading data... [Done]
TimeSeries([-4.70701276e-19, -4.77519734e-19, -4.70838351e-19,
            ..., -3.65035985e-19, -3.80810766e-19,
            -3.83224728e-19]
           unit: dimensionless,
           t0: 1187008877.0 s,
           dt: 0.000244140625 s,
           name: Strain,
           channel: None)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import welch
from scipy.optimize import curve_fit
from gwpy.timeseries import TimeSeries

# Données fournies (TimeSeries)
# Convert the list to a numpy array to avoid copy issues
strain_data = TimeSeries(
    np.array([-4.70701276e-19, -4.77519734e-19, -4.70838351e-19, ..., -3.65035985e-19, -3.80810766e-19, -3.83224728e-19]), # Changed to np.array
    unit='dimensionless',
    t0=1187008877.0,
    dt=0.000244140625,
    name='Strain',
    channel=None
)

# ... rest of the code remains the same ...

In [ ]:
!pip install matplotlib==3.7.3

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import welch
from scipy.optimize import curve_fit
from gwpy.timeseries import TimeSeries

# Paramètres des données
n_samples = 40960  # 10 secondes à 4096 Hz
sampling_rate = 4096  # Hz
dt = 1 / sampling_rate  # Intervalle de temps entre échantillons

# Simuler des données de strain (car la liste fournie est incomplète)
# Remplacez cette ligne par vos données réelles si disponibles
np.random.seed(42)  # Pour la reproductibilité
strain_values = np.random.normal(loc=-4e-19, scale=2e-20, size=n_samples)  # Bruit centré autour de -4e-19

# Création de l’objet TimeSeries
strain_data = TimeSeries(
    strain_values,
    unit=None,
    t0=1187008877.0,
    dt=dt,
    name='Strain',
    channel=None
)

# Paramètres pour l’analyse
freq_min, freq_max = 10, 2000  # Plage de fréquences analysées (Hz), limitée par la fréquence de Nyquist (2048 Hz)

# Nettoyage des données : suppression des valeurs aberrantes
def clean_data(data, threshold=5):
    median = np.median(data)
    iqr = np.percentile(data, 75) - np.percentile(data, 25)
    lower_bound = median - threshold * iqr
    upper_bound = median + threshold * iqr
    cleaned_data = np.where((data < lower_bound) | (data > upper_bound), median, data)
    print(f"Nombre de valeurs aberrantes remplacées : {np.sum((data < lower_bound) | (data > upper_bound))}")
    return cleaned_data

# Modèle de bruit classique pour le strain : S(f) = A f^(-2) + B f^2 + C
def noise_model(f, A, B, C):
    return A * f**(-2) + B * f**2 + C

# Modèle alternatif pour détecter des anomalies : S(f) = A f^C + B
def anomaly_model(f, A, B, C):
    return A * f**C + B

# Création du rapport PDF
with PdfPages("rapport_analyse_bruit_gwosc_1187008877.pdf") as pdf:
    report_text = []

    # Extraction des données
    data = strain_data.value
    print(f"Données chargées : {data.size} échantillons")

    # Nettoyage des NaN et des valeurs aberrantes
    valid_data = data[~np.isnan(data)]
    cleaned_data = clean_data(valid_data)

    # Calcul de la densité spectrale de puissance (PSD) avec Welch
    frequencies, S_f = welch(cleaned_data, fs=sampling_rate, nperseg=1024, noverlap=512)

    # Filtrer les fréquences pertinentes (10-2000 Hz)
    mask = (frequencies >= freq_min) & (frequencies <= freq_max)
    frequencies = frequencies[mask]
    S_f = S_f[mask]

    # Normalisation de S(f) par la médiane
    S_f_norm = S_f / np.median(S_f)
    print(f"Valeur de normalisation (médiane) : {np.median(S_f)}")
    print(f"Valeur de normalisation (maximum) : {np.max(S_f)}")

    # Vérification des données avant le tracé
    if np.any(frequencies <= 0) or np.any(S_f_norm <= 0):
        raise ValueError("Les fréquences ou S_f_norm contiennent des valeurs non positives, ce qui est invalide pour un tracé logarithmique.")
    print(f"Min fréquence : {np.min(frequencies)}, Max fréquence : {np.max(frequencies)}")
    print(f"Min S_f_norm : {np.min(S_f_norm)}, Max S_f_norm : {np.max(S_f_norm)}")

    # Ajustement du modèle classique
    p0 = [1e-40, 1e-48, 1e-45]  # Valeurs initiales pour [A, B, C]
    bounds = ([0, 0, 0], [np.inf, np.inf, np.inf])
    try:
        popt, pcov = curve_fit(noise_model, frequencies, S_f_norm, p0=p0, bounds=bounds, maxfev=10000)
        A_fit, B_fit, C_fit = popt
        report_text.append(f"Modèle classique ajusté : S(f) = {A_fit:.2e} * f^(-2) + {B_fit:.2e} * f^2 + {C_fit:.2e}")
    except RuntimeError as e:
        print(f"Erreur dans l’ajustement classique : {e}")
        A_fit, B_fit, C_fit = 1e-40, 0, 1e-45
        report_text.append(f"Échec de l’ajustement classique, valeurs par défaut : S(f) = {A_fit:.2e} * f^(-2) + {B_fit:.2e} * f^2 + {C_fit:.2e}")

    # Calcul du spectre ajusté (modèle classique)
    S_f_background = noise_model(frequencies, A_fit, B_fit, C_fit)

    # Ajustement du modèle alternatif (pour détecter des anomalies)
    p0_alt = [1e-5, 1e-5, -1.0]  # Valeurs initiales ajustées
    bounds_alt = ([0, 0, -5], [np.inf, np.inf, 5])
    try:
        popt_alt, pcov_alt = curve_fit(anomaly_model, frequencies, S_f_norm, p0=p0_alt, bounds=bounds_alt, maxfev=10000)
        A_alt, B_alt, C_alt = popt_alt
        report_text.append(f"Modèle alternatif ajusté : S(f) = {A_alt:.2e} * f^{C_alt:.2f} + {B_alt:.2e}")
    except RuntimeError as e:
        print(f"Erreur dans l’ajustement alternatif : {e}")
        A_alt, B_alt, C_alt = 1e-5, 1e-5, 1.0
        report_text.append(f"Échec de l’ajustement alternatif, valeurs par défaut : S(f) = {A_alt:.2e} * f^{C_alt:.2f} + {B_alt:.2e}")

    # Calcul du spectre ajusté (modèle alternatif)
    S_f_anomaly = anomaly_model(frequencies, A_alt, B_alt, C_alt)

    # Bootstrap pour estimer les incertitudes
    n_bootstrap = 500
    bootstrap_psds = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(cleaned_data, size=len(cleaned_data), replace=True)
        _, psd_sample = welch(sample, fs=sampling_rate, nperseg=1024, noverlap=512)
        psd_sample = psd_sample[mask] / np.median(psd_sample[mask])
        bootstrap_psds.append(psd_sample)
    bootstrap_psds = np.array(bootstrap_psds)
    psd_mean = np.mean(bootstrap_psds, axis=0)
    psd_std = np.std(bootstrap_psds, axis=0)
    psd_std = np.where(psd_std == 0, 1e-10, psd_std)

    # Calcul du chi2 pour le modèle classique
    chi2_classic = np.sum(((S_f_norm - S_f_background) / psd_std) ** 2)
    report_text.append(f"Chi2 modèle classique : {chi2_classic:.2e}")

    # Calcul du chi2 pour le modèle alternatif
    chi2_anomaly = np.sum(((S_f_norm - S_f_anomaly) / psd_std) ** 2)
    report_text.append(f"Chi2 modèle alternatif : {chi2_anomaly:.2e}")

    # Simulation Monte Carlo pour la p-valeur (modèle classique)
    n_simulations = 100
    chi2_simulated = []
    for _ in range(n_simulations):
        noise = np.random.normal(0, 1, size=len(cleaned_data)) * np.sqrt(np.mean(S_f))
        _, psd_sim = welch(noise, fs=sampling_rate, nperseg=1024, noverlap=512)
        psd_sim = psd_sim[mask] / np.median(psd_sim[mask])
        chi2_val = np.sum(((psd_sim - psd_mean) / psd_std) ** 2)
        chi2_simulated.append(chi2_val)
    p_value_classic = np.mean(np.array(chi2_simulated) > chi2_classic)
    report_text.append(f"P-valeur modèle classique : {p_value_classic:.3f}")

    # Graphique 1 : Spectre observé vs ajusté (approche explicite)
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.loglog(frequencies, S_f_norm, label="S(f) observé (normalisé)")
    ax.loglog(frequencies, S_f_background, label="S(f) classique ajusté", linestyle="--")
    ax.loglog(frequencies, S_f_anomaly, label="S(f) alternatif ajusté", linestyle=":")
    ax.fill_between(frequencies, psd_mean - 2*psd_std, psd_mean + 2*psd_std, alpha=0.2, label="Intervalle 95% (Bootstrap)")
    ax.set_xlabel("Fréquence (Hz)")
    ax.set_ylabel("Densité spectrale S(f) (normalisée)")
    ax.set_title("Spectre de bruit (GWOSC, 10s, 1187008877)")
    ax.legend()
    ax.grid(True, which="both", ls="--")
    pdf.savefig(fig)
    plt.close(fig)

    # Graphique 2 : Série temporelle (pour rechercher un signal d’onde gravitationnelle)
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(strain_data.times.value, strain_data.value, label="Strain")
    ax.set_xlabel("Temps (s)")
    ax.set_ylabel("Strain")
    ax.set_title("Série temporelle (GWOSC, 10s, 1187008877)")
    ax.legend()
    ax.grid(True)
    pdf.savefig(fig)
    plt.close(fig)

    # Page texte du rapport
    fig, ax = plt.subplots(figsize=(8, 11))
    ax.text(0.1, 0.95, "Rapport d’analyse (GWOSC, 1187008877)", fontsize=16, fontweight='bold')
    ax.text(0.1, 0.9, "\n".join(report_text), fontsize=12, verticalalignment='top')
    ax.axis('off')
    pdf.savefig(fig)
    plt.close(fig)

    # Affichage console
    print("\n=== Rapport d’analyse ===")
    for line in report_text:
        print(line)
    print("=== Fin du Rapport ===")

Données chargées : 40960 échantillons
Nombre de valeurs aberrantes remplacées : 0
Valeur de normalisation (médiane) : 1.9447821116294995e-43
Valeur de normalisation (maximum) : 2.6745767256730534e-43
Min fréquence : 12.0, Max fréquence : 2000.0
Min S_f_norm : 0.6733420190935852, Max S_f_norm : 1.3752577780716377

=== Rapport d’analyse ===
Modèle classique ajusté : S(f) = 2.78e-14 * f^(-2) + 1.73e-09 * f^2 + 1.00e+00
Modèle alternatif ajusté : S(f) = 1.08e-12 * f^-2.94 + 1.00e+00
Chi2 modèle classique : 4.96e+02
Chi2 modèle alternatif : 4.96e+02
P-valeur modèle classique : 0.590
=== Fin du Rapport ===


In [15]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import welch
from scipy.optimize import curve_fit
from gwpy.timeseries import TimeSeries

# Paramètres des données
n_samples = 40960  # 10 secondes à 4096 Hz
sampling_rate = 4096  # Hz
dt = 1 / sampling_rate  # Intervalle de temps entre échantillons

# Simuler des données de strain (car la liste fournie est incomplète)
# Remplacez cette ligne par vos données réelles si disponibles
np.random.seed(42)  # Pour la reproductibilité
strain_values = np.random.normal(loc=-4e-19, scale=2e-20, size=n_samples)  # Bruit centré autour de -4e-19

# Création de l’objet TimeSeries
strain_data = TimeSeries(
    strain_values,
    unit=None,
    t0=1187008877.0,
    dt=dt,
    name='Strain',
    channel=None
)

# Paramètres pour l’analyse
freq_min, freq_max = 10, 2000  # Plage de fréquences analysées (Hz), limitée par la fréquence de Nyquist (2048 Hz)

# Nettoyage des données : suppression des valeurs aberrantes
def clean_data(data, threshold=5):
    median = np.median(data)
    iqr = np.percentile(data, 75) - np.percentile(data, 25)
    lower_bound = median - threshold * iqr
    upper_bound = median + threshold * iqr
    cleaned_data = np.where((data < lower_bound) | (data > upper_bound), median, data)
    print(f"Nombre de valeurs aberrantes remplacées : {np.sum((data < lower_bound) | (data > upper_bound))}")
    return cleaned_data

# Modèle de bruit classique pour le strain : S(f) = A f^(-2) + B f^2 + C
def noise_model(f, A, B, C):
    return A * f**(-2) + B * f**2 + C

# Modèle alternatif pour détecter des anomalies : S(f) = A f^C + B
def anomaly_model(f, A, B, C):
    return A * f**C + B

# Création du rapport PDF
with PdfPages("rapport_analyse_bruit_gwosc_1187008877.pdf") as pdf:
    report_text = []

    # Extraction des données
    data = strain_data.value
    print(f"Données chargées : {data.size} échantillons")

    # Nettoyage des NaN et des valeurs aberrantes
    valid_data = data[~np.isnan(data)]
    cleaned_data = clean_data(valid_data)

    # Calcul de la densité spectrale de puissance (PSD) avec Welch
    frequencies, S_f = welch(cleaned_data, fs=sampling_rate, nperseg=1024, noverlap=512)

    # Filtrer les fréquences pertinentes (10-2000 Hz)
    mask = (frequencies >= freq_min) & (frequencies <= freq_max)
    frequencies = frequencies[mask]
    S_f = S_f[mask]

    # Normalisation de S(f) par la médiane
    S_f_norm = S_f / np.median(S_f)
    print(f"Valeur de normalisation (médiane) : {np.median(S_f)}")
    print(f"Valeur de normalisation (maximum) : {np.max(S_f)}")

    # Vérification des données avant le tracé
    if np.any(frequencies <= 0) or np.any(S_f_norm <= 0):
        raise ValueError("Les fréquences ou S_f_norm contiennent des valeurs non positives, ce qui est invalide pour un tracé logarithmique.")
    print(f"Min fréquence : {np.min(frequencies)}, Max fréquence : {np.max(frequencies)}")
    print(f"Min S_f_norm : {np.min(S_f_norm)}, Max S_f_norm : {np.max(S_f_norm)}")

    # Ajustement du modèle classique
    p0 = [1e-40, 1e-48, 1e-45]  # Valeurs initiales pour [A, B, C]
    bounds = ([0, 0, 0], [np.inf, np.inf, np.inf])
    try:
        popt, pcov = curve_fit(noise_model, frequencies, S_f_norm, p0=p0, bounds=bounds, maxfev=10000)
        A_fit, B_fit, C_fit = popt
        report_text.append(f"Modèle classique ajusté : S(f) = {A_fit:.2e} * f^(-2) + {B_fit:.2e} * f^2 + {C_fit:.2e}")
    except RuntimeError as e:
        print(f"Erreur dans l’ajustement classique : {e}")
        A_fit, B_fit, C_fit = 1e-40, 0, 1e-45
        report_text.append(f"Échec de l’ajustement classique, valeurs par défaut : S(f) = {A_fit:.2e} * f^(-2) + {B_fit:.2e} * f^2 + {C_fit:.2e}")

    # Calcul du spectre ajusté (modèle classique)
    S_f_background = noise_model(frequencies, A_fit, B_fit, C_fit)

    # Ajustement du modèle alternatif (pour détecter des anomalies)
    p0_alt = [1e-5, 1e-5, -1.0]  # Valeurs initiales ajustées
    bounds_alt = ([0, 0, -5], [np.inf, np.inf, 5])
    try:
        popt_alt, pcov_alt = curve_fit(anomaly_model, frequencies, S_f_norm, p0=p0_alt, bounds=bounds_alt, maxfev=10000)
        A_alt, B_alt, C_alt = popt_alt
        report_text.append(f"Modèle alternatif ajusté : S(f) = {A_alt:.2e} * f^{C_alt:.2f} + {B_alt:.2e}")
    except RuntimeError as e:
        print(f"Erreur dans l’ajustement alternatif : {e}")
        A_alt, B_alt, C_alt = 1e-5, 1e-5, 1.0
        report_text.append(f"Échec de l’ajustement alternatif, valeurs par défaut : S(f) = {A_alt:.2e} * f^{C_alt:.2f} + {B_alt:.2e}")

    # Calcul du spectre ajusté (modèle alternatif)
    S_f_anomaly = anomaly_model(frequencies, A_alt, B_alt, C_alt)

    # Bootstrap pour estimer les incertitudes
    n_bootstrap = 500
    bootstrap_psds = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(cleaned_data, size=len(cleaned_data), replace=True)
        _, psd_sample = welch(sample, fs=sampling_rate, nperseg=1024, noverlap=512)
        psd_sample = psd_sample[mask] / np.median(psd_sample[mask])
        bootstrap_psds.append(psd_sample)
    bootstrap_psds = np.array(bootstrap_psds)
    psd_mean = np.mean(bootstrap_psds, axis=0)
    psd_std = np.std(bootstrap_psds, axis=0)
    psd_std = np.where(psd_std == 0, 1e-10, psd_std)

    # Calcul du chi2 pour le modèle classique
    chi2_classic = np.sum(((S_f_norm - S_f_background) / psd_std) ** 2)
    report_text.append(f"Chi2 modèle classique : {chi2_classic:.2e}")

    # Calcul du chi2 pour le modèle alternatif
    chi2_anomaly = np.sum(((S_f_norm - S_f_anomaly) / psd_std) ** 2)
    report_text.append(f"Chi2 modèle alternatif : {chi2_anomaly:.2e}")

    # Simulation Monte Carlo pour la p-valeur (modèle classique)
    n_simulations = 100
    chi2_simulated = []
    for _ in range(n_simulations):
        noise = np.random.normal(0, 1, size=len(cleaned_data)) * np.sqrt(np.mean(S_f))
        _, psd_sim = welch(noise, fs=sampling_rate, nperseg=1024, noverlap=512)
        psd_sim = psd_sim[mask] / np.median(psd_sim[mask])
        chi2_val = np.sum(((psd_sim - psd_mean) / psd_std) ** 2)
        chi2_simulated.append(chi2_val)
    p_value_classic = np.mean(np.array(chi2_simulated) > chi2_classic)
    report_text.append(f"P-valeur modèle classique : {p_value_classic:.3f}")

    # Graphique 1 : Spectre observé vs ajusté (approche explicite)
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.loglog(frequencies, S_f_norm, label="S(f) observé (normalisé)")
    ax.loglog(frequencies, S_f_background, label="S(f) classique ajusté", linestyle="--")
    ax.loglog(frequencies, S_f_anomaly, label="S(f) alternatif ajusté", linestyle=":")
    ax.fill_between(frequencies, psd_mean - 2*psd_std, psd_mean + 2*psd_std, alpha=0.2, label="Intervalle 95% (Bootstrap)")
    ax.set_xlabel("Fréquence (Hz)")
    ax.set_ylabel("Densité spectrale S(f) (normalisée)")
    ax.set_title("Spectre de bruit (GWOSC, 10s, 1187008877)")
    ax.legend()
    ax.grid(True, which="both", ls="--")
    pdf.savefig(fig)
    plt.close(fig)

    # Graphique 2 : Série temporelle (pour rechercher un signal d’onde gravitationnelle)
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(strain_data.times.value, strain_data.value, label="Strain")
    ax.set_xlabel("Temps (s)")
    ax.set_ylabel("Strain")
    ax.set_title("Série temporelle (GWOSC, 10s, 1187008877)")
    ax.legend()
    ax.grid(True)
    pdf.savefig(fig)
    plt.close(fig)

    # Page texte du rapport
    fig, ax = plt.subplots(figsize=(8, 11))
    ax.text(0.1, 0.95, "Rapport d’analyse (GWOSC, 1187008877)", fontsize=16, fontweight='bold')
    ax.text(0.1, 0.9, "\n".join(report_text), fontsize=12, verticalalignment='top')
    ax.axis('off')
    pdf.savefig(fig)
    plt.close(fig)

    # Affichage console
    print("\n=== Rapport d’analyse ===")
    for line in report_text:
        print(line)
    print("=== Fin du Rapport ===")

Données chargées : 40960 échantillons
Nombre de valeurs aberrantes remplacées : 0
Valeur de normalisation (médiane) : 1.9447821116294995e-43
Valeur de normalisation (maximum) : 2.6745767256730534e-43
Min fréquence : 12.0, Max fréquence : 2000.0
Min S_f_norm : 0.6733420190935852, Max S_f_norm : 1.3752577780716377

=== Rapport d’analyse ===
Modèle classique ajusté : S(f) = 2.78e-14 * f^(-2) + 1.73e-09 * f^2 + 1.00e+00
Modèle alternatif ajusté : S(f) = 1.08e-12 * f^-2.94 + 1.00e+00
Chi2 modèle classique : 4.96e+02
Chi2 modèle alternatif : 4.96e+02
P-valeur modèle classique : 0.590
=== Fin du Rapport ===


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import welch
from scipy.optimize import curve_fit
from gwpy.timeseries import TimeSeries

# Paramètres des données
n_samples = 40960  # 10 secondes à 4096 Hz
sampling_rate = 4096  # Hz
dt = 1 / sampling_rate  # Intervalle de temps entre échantillons

# Simuler des données de strain (car la liste fournie est incomplète)
# Remplacez cette ligne par vos données réelles si disponibles
np.random.seed(42)  # Pour la reproductibilité
strain_values = np.random.normal(loc=-4e-19, scale=2e-20, size=n_samples)  # Bruit centré autour de -4e-19

# Création de l’objet TimeSeries
strain_data = TimeSeries(
    strain_values,
    unit=None,
    t0=1187008877.0,
    dt=dt,
    name='Strain',
    channel=None
)

# Paramètres pour l’analyse
freq_min, freq_max = 10, 2000  # Plage de fréquences analysées (Hz), limitée par la fréquence de Nyquist (2048 Hz)

# Nettoyage des données : suppression des valeurs aberrantes
def clean_data(data, threshold=5):
    median = np.median(data)
    iqr = np.percentile(data, 75) - np.percentile(data, 25)
    lower_bound = median - threshold * iqr
    upper_bound = median + threshold * iqr
    cleaned_data = np.where((data < lower_bound) | (data > upper_bound), median, data)
    print(f"Nombre de valeurs aberrantes remplacées : {np.sum((data < lower_bound) | (data > upper_bound))}")
    return cleaned_data

# Modèle de bruit classique pour le strain : S(f) = A f^(-2) + B f^2 + C
def noise_model(f, A, B, C):
    return A * f**(-2) + B * f**2 + C

# Modèle alternatif pour détecter des anomalies : S(f) = A f^C + B
def anomaly_model(f, A, B, C):
    return A * f**C + B

# Création du rapport PDF
with PdfPages("rapport_analyse_bruit_gwosc_1187008877.pdf") as pdf:
    report_text = []

    # Extraction des données
    data = strain_data.value
    print(f"Données chargées : {data.size} échantillons")

    # Nettoyage des NaN et des valeurs aberrantes
    valid_data = data[~np.isnan(data)]
    cleaned_data = clean_data(valid_data)

    # Calcul de la densité spectrale de puissance (PSD) avec Welch
    frequencies, S_f = welch(cleaned_data, fs=sampling_rate, nperseg=1024, noverlap=512)

    # Filtrer les fréquences pertinentes (10-2000 Hz)
    mask = (frequencies >= freq_min) & (frequencies <= freq_max)
    frequencies = frequencies[mask]
    S_f = S_f[mask]

    # Normalisation de S(f) par la médiane
    S_f_norm = S_f / np.median(S_f)
    print(f"Valeur de normalisation (médiane) : {np.median(S_f)}")
    print(f"Valeur de normalisation (maximum) : {np.max(S_f)}")

    # Vérification des données avant le tracé
    if np.any(frequencies <= 0) or np.any(S_f_norm <= 0):
        raise ValueError("Les fréquences ou S_f_norm contiennent des valeurs non positives, ce qui est invalide pour un tracé logarithmique.")

    # Ajustement du modèle classique
    p0 = [1e-40, 1e-48, 1e-45]  # Valeurs initiales pour [A, B, C]
    bounds = ([0, 0, 0], [np.inf, np.inf, np.inf])
    try:
        popt, pcov = curve_fit(noise_model, frequencies, S_f_norm, p0=p0, bounds=bounds, maxfev=10000)
        A_fit, B_fit, C_fit = popt
        report_text.append(f"Modèle classique ajusté : S(f) = {A_fit:.2e} * f^(-2) + {B_fit:.2e} * f^2 + {C_fit:.2e}")
    except RuntimeError as e:
        print(f"Erreur dans l’ajustement classique : {e}")
        A_fit, B_fit, C_fit = 1e-40, 0, 1e-45
        report_text.append(f"Échec de l’ajustement classique, valeurs par défaut : S(f) = {A_fit:.2e} * f^(-2) + {B_fit:.2e} * f^2 + {C_fit:.2e}")

    # Calcul du spectre ajusté (modèle classique)
    S_f_background = noise_model(frequencies, A_fit, B_fit, C_fit)

    # Ajustement du modèle alternatif (pour détecter des anomalies)
    p0_alt = [1e-5, 1e-5, -1.0]  # Valeurs initiales ajustées
    bounds_alt = ([0, 0, -5], [np.inf, np.inf, 5])
    try:
        popt_alt, pcov_alt = curve_fit(anomaly_model, frequencies, S_f_norm, p0=p0_alt, bounds=bounds_alt, maxfev=10000)
        A_alt, B_alt, C_alt = popt_alt
        report_text.append(f"Modèle alternatif ajusté : S(f) = {A_alt:.2e} * f^{C_alt:.2f} + {B_alt:.2e}")
    except RuntimeError as e:
        print(f"Erreur dans l’ajustement alternatif : {e}")
        A_alt, B_alt, C_alt = 1e-5, 1e-5, 1.0
        report_text.append(f"Échec de l’ajustement alternatif, valeurs par défaut : S(f) = {A_alt:.2e} * f^{C_alt:.2f} + {B_alt:.2e}")

    # Calcul du spectre ajusté (modèle alternatif)
    S_f_anomaly = anomaly_model(frequencies, A_alt, B_alt, C_alt)

    # Bootstrap pour estimer les incertitudes
    n_bootstrap = 500
    bootstrap_psds = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(cleaned_data, size=len(cleaned_data), replace=True)
        _, psd_sample = welch(sample, fs=sampling_rate, nperseg=1024, noverlap=512)
        psd_sample = psd_sample[mask] / np.median(psd_sample[mask])
        bootstrap_psds.append(psd_sample)
    bootstrap_psds = np.array(bootstrap_psds)
    psd_mean = np.mean(bootstrap_psds, axis=0)
    psd_std = np.std(bootstrap_psds, axis=0)
    psd_std = np.where(psd_std == 0, 1e-10, psd_std)

    # Calcul du chi2 pour le modèle classique
    chi2_classic = np.sum(((S_f_norm - S_f_background) / psd_std) ** 2)
    report_text.append(f"Chi2 modèle classique : {chi2_classic:.2e}")

    # Calcul du chi2 pour le modèle alternatif
    chi2_anomaly = np.sum(((S_f_norm - S_f_anomaly) / psd_std) ** 2)
    report_text.append(f"Chi2 modèle alternatif : {chi2_anomaly:.2e}")

    # Simulation Monte Carlo pour la p-valeur (modèle classique)
    n_simulations = 100
    chi2_simulated = []
    for _ in range(n_simulations):
        noise = np.random.normal(0, 1, size=len(cleaned_data)) * np.sqrt(np.mean(S_f))
        _, psd_sim = welch(noise, fs=sampling_rate, nperseg=1024, noverlap=512)
        psd_sim = psd_sim[mask] / np.median(psd_sim[mask])
        chi2_val = np.sum(((psd_sim - psd_mean) / psd_std) ** 2)
        chi2_simulated.append(chi2_val)
    p_value_classic = np.mean(np.array(chi2_simulated) > chi2_classic)
    report_text.append(f"P-valeur modèle classique : {p_value_classic:.3f}")

    # Graphique 1 : Spectre observé vs ajusté
    plt.figure(figsize=(12, 6))
    plt.loglog(frequencies, S_f_norm, label="S(f) observé (normalisé)")
    plt.loglog(frequencies, S_f_background, label="S(f) classique ajusté", linestyle="--")
    plt.loglog(frequencies, S_f_anomaly, label="S(f) alternatif ajusté", linestyle=":")
    plt.fill_between(frequencies, psd_mean - 2*psd_std, psd_mean + 2*psd_std, alpha=0.2, label="Intervalle 95% (Bootstrap)")
    plt.xlabel("Fréquence (Hz)")
    plt.ylabel("Densité spectrale S(f) (normalisée)")
    plt.title("Spectre de bruit (GWOSC, 10s, 1187008877)")
    plt.legend()
    plt.grid(True, which="both", ls="--")
    pdf.savefig()
    plt.close()

    # Graphique 2 : Série temporelle (pour rechercher un signal d’onde gravitationnelle)
    plt.figure(figsize=(12, 6))
    plt.plot(strain_data.times.value, strain_data.value, label="Strain")
    plt.xlabel("Temps (s)")
    plt.ylabel("Strain")
    plt.title("Série temporelle (GWOSC, 10s, 1187008877)")
    plt.legend()
    plt.grid(True)
    pdf.savefig()
    plt.close()

    # Page texte du rapport
    plt.figure(figsize=(8, 11))
    plt.text(0.1, 0.95, "Rapport d’analyse (GWOSC, 1187008877)", fontsize=16, fontweight='bold')
    plt.text(0.1, 0.9, "\n".join(report_text), fontsize=12, verticalalignment='top')
    plt.axis('off')
    pdf.savefig()
    plt.close()

    # Affichage console
    print("\n=== Rapport d’analyse ===")
    for line in report_text:
        print(line)
    print("=== Fin du Rapport ===")

Données chargées : 40960 échantillons
Nombre de valeurs aberrantes remplacées : 0
Valeur de normalisation (médiane) : 1.9447821116294995e-43
Valeur de normalisation (maximum) : 2.6745767256730534e-43

=== Rapport d’analyse ===
Modèle classique ajusté : S(f) = 2.78e-14 * f^(-2) + 1.73e-09 * f^2 + 1.00e+00
Modèle alternatif ajusté : S(f) = 1.08e-12 * f^-2.94 + 1.00e+00
Chi2 modèle classique : 4.96e+02
Chi2 modèle alternatif : 4.96e+02
P-valeur modèle classique : 0.590
=== Fin du Rapport ===


In [14]:
import requests
from gwpy.timeseries import TimeSeries


def fetch_run_gps_times(run):
    "Fetch gwosc archive and return the (start, end) GPS time tuple of the run."
    response = requests.get("https://gwosc.org/archive/all/json/").json()
    runs = response["runs"]
    run_info = runs.get(run)
    if run_info is None:
        raise ValueError(f"Could not find run {run}. Available runs: {runs.keys()}")
    return run_info["GPSstart"], run_info["GPSend"]


def fetch_strain_list(run, detector, gps_start=None, gps_end=None):
    "Return the list of strain file info for `run` and `detector`."
    if gps_start is None or gps_end is None:
        start, end = fetch_run_gps_times(run)
        gps_start = gps_start or start
        gps_end = gps_end or end

    # Get the strain list
    fetch_url = (
        f"https://gwosc.org/archive/links/"
        f"{run}/{detector}/{gps_start}/{gps_end}/json/"
    )
    response = requests.get(fetch_url)
    response.raise_for_status()
    return response.json()["strain"]


def download_strain_file(download_url):
    "Download the strain file on the given url and save to disk."
    # In the next line I parse the file name from the download url.
    # Ideally, the file name should be grabbed from the
    # Content-Disposition response header.
    filename = download_url.split("/")[-1]
    with requests.get(download_url, stream=True) as r:
        r.raise_for_status()
        with open(filename, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return filename


def main():
    strain_files = fetch_strain_list("O3b_4KHZ_R1", "H1")
    try:
        with open("filesdone.txt", "r") as fp:
            donelist = [f.strip() for f in fp.readlines()]
    except FileNotFoundError:
        donelist = []
    for afile in strain_files:
        if afile["url"] in donelist:
            continue
        if afile["format"] == "hdf5":
            print(f"Downloading {afile['url']}")
            fname = download_strain_file(afile["url"])
            tseries = TimeSeries.read(fname, format="hdf5.gwosc")
            with open("filesdone.txt", "a") as fp:
                fp.write(f"{afile['url']}\n")
            # process tseries here


if __name__ == "__main__":
    main()


KeyboardInterrupt: 

In [ ]:
import h5py
import requests
from io import BytesIO
import numpy as np

# Liste d'URLs (par exemple, les 3 premières pour tester)
urls = [
    "http://gwosc.org/archive/data/O3b_4KHZ_R1/1256194048/H-H1_GWOSC_O3b_4KHZ_R1-1256660992-4096.hdf5",
    "http://gwosc.org/archive/data/O3b_4KHZ_R1/1256194048/H-H1_GWOSC_O3b_4KHZ_R1-1256665088-4096.hdf5",
    "http://gwosc.org/archive/data/O3b_4KHZ_R1/1256194048/H-H1_GWOSC_O3b_4KHZ_R1-1256669184-4096.hdf5"
]

# Fonction pour lire un fichier HDF5 depuis une URL
def read_hdf5_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        file_like_object = BytesIO(response.content)
        with h5py.File(file_like_object, 'r') as f:
            if 'strain' in f and 'Strain' in f['strain']:
                data = f['strain/Strain'][:]
                print(f"Données lues depuis {url} : {data.size} échantillons")
                return data
            else:
                print(f"Dataset 'strain/Strain' non trouvé dans {url}")
                return None
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors du téléchargement de {url} : {e}")
        return None
    except Exception as e:
        print(f"Erreur inattendue pour {url} : {e}")
        return None

# Lire et combiner les données de plusieurs fichiers
combined_data = []
for url in urls:
    data = read_hdf5_from_url(url)
    if data is not None:
        combined_data.append(data)

# Concaténer les données si elles existent
if combined_data:
    combined_data = np.concatenate(combined_data)
    print("Attribution : Data from GWOSC, released under CC BY 4.0 License")
    print("Taille totale des données combinées :", combined_data.size)
    print("Échantillon des premières valeurs :", combined_data[:10])
else:
    print("Aucune donnée n'a pu être lue.")

In [ ]:
import h5py
import requests
from io import BytesIO
import numpy as np

# Liste des URLs
urls = [
    "http://gwosc.org/archive/data/O3b_4KHZ_R1/1256194048/H-H1_GWOSC_O3b_4KHZ_R1-1256660992-4096.hdf5",
    "http://gwosc.org/archive/data/O3b_4KHZ_R1/1256194048/H-H1_GWOSC_O3b_4KHZ_R1-1256665088-4096.hdf5",
    "http://gwosc.org/archive/data/O3b_4KHZ_R1/1256194048/H-H1_GWOSC_O3b_4KHZ_R1-1256669184-4096.hdf5"
]

# Fonction pour lire un fichier HDF5 depuis une URL
def read_hdf5_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        file_like_object = BytesIO(response.content)
        with h5py.File(file_like_object, 'r') as f:
            if 'strain' in f and 'Strain' in f['strain']:
                data = f['strain/Strain'][:]
                print(f"Données lues depuis {url} : {data.size} échantillons")
                return data
            else:
                print(f"Dataset 'strain/Strain' non trouvé dans {url}")
                return None
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors du téléchargement de {url} : {e}")
        return None
    except Exception as e:
        print(f"Erreur inattendue pour {url} : {e}")
        return None

# Lire et combiner les données
combined_data = []
for url in urls:
    data = read_hdf5_from_url(url)
    if data is not None:
        combined_data.append(data)

# Vérifications sur les données combinées
if combined_data:
    combined_data = np.concatenate(combined_data)
    print("Attribution : Data from GWOSC, released under CC BY 4.0 License")
    print("Taille totale des données combinées :", combined_data.size)

    # Vérifier la proportion de NaN
    nan_count = np.isnan(combined_data).sum()
    print(f"Nombre de valeurs NaN : {nan_count} ({100 * nan_count / combined_data.size:.2f}%)")

    # Afficher des échantillons à différents endroits
    print("Échantillon des premières valeurs :", combined_data[:10])
    print("Échantillon au milieu :", combined_data[combined_data.size//2:combined_data.size//2 + 10])
    print("Échantillon des dernières valeurs :", combined_data[-10:])

    # Si tout est NaN, arrêter ici
    if nan_count == combined_data.size:
        print("Toutes les données sont NaN. Vérifiez les fichiers ou la structure des données.")
    else:
        # Trouver la première valeur non-NaN pour inspection
        non_nan_indices = np.where(~np.isnan(combined_data))[0]
        if len(non_nan_indices) > 0:
            first_non_nan_idx = non_nan_indices[0]
            print(f"Première valeur non-NaN à l'index {first_non_nan_idx} : {combined_data[first_non_nan_idx]}")
else:
    print("Aucune donnée n'a pu être lue.")

In [ ]:
# Filtrer les NaN
valid_data = combined_data[~np.isnan(combined_data)]
print("Taille des données valides :", valid_data.size)
print("Échantillon des premières valeurs valides :", valid_data[:10])

# Calculer la durée des données valides (en secondes)
sampling_rate = 4000  # 4 kHz
duration_valid = valid_data.size / sampling_rate
print(f"Durée des données valides : {duration_valid:.2f} secondes")
# Timestamp de début du premier fichier
start_time_gps = 1256660992

# Calculer le temps correspondant au premier échantillon non-NaN
nan_duration = 10039296 / sampling_rate  # Durée des NaN en secondes
start_valid_time_gps = start_time_gps + nan_duration
print(f"Les données valides commencent à GPS : {start_valid_time_gps:.2f}")

# Créer un tableau de temps pour les données valides
time_valid = np.arange(0, valid_data.size) / sampling_rate + start_valid_time_gps
print("Échantillon des premiers timestamps :", time_valid[:10])
import matplotlib.pyplot as plt

# Tracer une petite portion des données valides (par exemple, 1 seconde)
plt.figure(figsize=(10, 4))
plt.plot(time_valid[:4000], valid_data[:4000], label="Strain H1")
plt.xlabel("Temps GPS (s)")
plt.ylabel("Strain")
plt.title("Données de déformation (H1) - 1 seconde")
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch
from scipy.stats import chi2

# Données valides à partir de votre sortie précédente
sampling_rate = 4000  # 4 kHz
valid_data = combined_data[~np.isnan(combined_data)]  # 40 292 352 échantillons
start_valid_time_gps = 1256663501.82
time_valid = np.arange(0, valid_data.size) / sampling_rate + start_valid_time_gps

# 1. Calculer le spectre de bruit S(f) avec la méthode de Welch
frequencies, psd = welch(valid_data, fs=sampling_rate, nperseg=4096, noverlap=2048)
S_f = psd  # Densité spectrale de puissance

# 2. Bootstrap pour estimer la variabilité de S(f)
n_bootstrap = 500  # Réduit pour accélérer, augmentez si nécessaire
bootstrap_psds = []
for _ in range(n_bootstrap):
    sample = np.random.choice(valid_data, size=len(valid_data), replace=True)
    _, psd_sample = welch(sample, fs=sampling_rate, nperseg=4096, noverlap=2048)
    bootstrap_psds.append(psd_sample)

bootstrap_psds = np.array(bootstrap_psds)
psd_mean = np.mean(bootstrap_psds, axis=0)
psd_std = np.std(bootstrap_psds, axis=0)
psd_ci_lower = np.percentile(bootstrap_psds, 2.5, axis=0)  # 95% CI
psd_ci_upper = np.percentile(bootstrap_psds, 97.5, axis=0)

# 3. Simulations Monte Carlo pour \(\chi^2\)
n_simulations = 500  # Réduit pour accélérer
chi2_simulated = []
for _ in range(n_simulations):
    noise = np.random.normal(0, 1, size=len(valid_data)) * np.sqrt(np.mean(S_f))
    _, psd_sim = welch(noise, fs=sampling_rate, nperseg=4096, noverlap=2048)
    chi2_val = np.sum(((psd_sim - psd_mean) / psd_std) ** 2)  # \(\chi^2\) simulé
    chi2_simulated.append(chi2_val)

# 4. Comparaison avec \(\chi^2 \sim 10^{21}\)
chi2_observed = 1e21  # Votre valeur observée
p_value_mc = np.mean(np.array(chi2_simulated) > chi2_observed)
print(f"P-valeur Monte Carlo pour \(\chi^2 = {chi2_observed}\): {p_value_mc}")

# Visualisation du spectre
plt.figure(figsize=(12, 6))
plt.loglog(frequencies, S_f, label="S(f) observé")
plt.fill_between(frequencies, psd_ci_lower, psd_ci_upper, alpha=0.2, label="Intervalle de confiance 95% (Bootstrap)")
plt.xlabel("Fréquence (Hz)")
plt.ylabel("Densité spectrale de puissance S(f)")
plt.title("Spectre de bruit (H1, GPS 1256663501.82 et suivants)")
plt.legend()
plt.grid(True, which="both", ls="--")
plt.show()

# Histogramme des \(\chi^2\) simulés
plt.figure(figsize=(10, 4))
plt.hist(chi2_simulated, bins=50, density=True, alpha=0.7, label="\(\chi^2\) simulés")
plt.axvline(chi2_observed, color="red", linestyle="--", label=f"\(\chi^2\) observé = {chi2_observed:.2e}")
plt.xlabel("\(\chi^2\)")
plt.ylabel("Densité")
plt.title("Distribution des \(\chi^2\) simulés")
plt.legend()
plt.show()

# Vérification des écarts extrêmes
max_chi2_simulated = np.max(chi2_simulated)
print(f"Maximum \(\chi^2\) simulé : {max_chi2_simulated:.2e}")
if max_chi2_simulated < chi2_observed:
    print("Le \(\chi^2\) observé dépasse largement les fluctuations statistiques simulées.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch
from scipy.stats import chi2

# Données valides
sampling_rate = 4000  # 4 kHz
valid_data = combined_data[~np.isnan(combined_data)]  # 40 292 352 échantillons

# Réduire la taille des données (par exemple, premières 1000 secondes = 4 000 000 échantillons)
subset_size = 4000000
valid_data_subset = valid_data[:subset_size]
start_valid_time_gps = 1256663501.82
time_valid = np.arange(0, valid_data_subset.size) / sampling_rate + start_valid_time_gps

# 1. Calculer le spectre de bruit S(f)
frequencies, psd = welch(valid_data_subset, fs=sampling_rate, nperseg=1024, noverlap=512)
S_f = psd

# 2. Bootstrap (réduit à 100 itérations)
n_bootstrap = 100
bootstrap_psds = []
for _ in range(n_bootstrap):
    sample = np.random.choice(valid_data_subset, size=len(valid_data_subset), replace=True)
    _, psd_sample = welch(sample, fs=sampling_rate, nperseg=1024, noverlap=512)
    bootstrap_psds.append(psd_sample)

bootstrap_psds = np.array(bootstrap_psds)
psd_mean = np.mean(bootstrap_psds, axis=0)
psd_std = np.std(bootstrap_psds, axis=0)
psd_ci_lower = np.percentile(bootstrap_psds, 2.5, axis=0)
psd_ci_upper = np.percentile(bootstrap_psds, 97.5, axis=0)

# 3. Simulations Monte Carlo (réduit à 100 itérations)
n_simulations = 100
chi2_simulated = []
for _ in range(n_simulations):
    noise = np.random.normal(0, 1, size=len(valid_data_subset)) * np.sqrt(np.mean(S_f))
    _, psd_sim = welch(noise, fs=sampling_rate, nperseg=1024, noverlap=512)
    chi2_val = np.sum(((psd_sim - psd_mean) / psd_std) ** 2)
    chi2_simulated.append(chi2_val)

# 4. Comparaison avec \(\chi^2 \sim 10^{21}\)
chi2_observed = 1e21
p_value_mc = np.mean(np.array(chi2_simulated) > chi2_observed)
print(f"P-valeur Monte Carlo pour \(\chi^2 = {chi2_observed}\): {p_value_mc}")
print(f"Maximum \(\chi^2\) simulé : {np.max(chi2_simulated):.2e}")

# Visualisation
plt.figure(figsize=(12, 6))
plt.loglog(frequencies, S_f, label="S(f) observé (1000s)")
plt.fill_between(frequencies, psd_ci_lower, psd_ci_upper, alpha=0.2, label="Intervalle de confiance 95%")
plt.xlabel("Fréquence (Hz)")
plt.ylabel("Densité spectrale de puissance S(f)")
plt.title("Spectre de bruit (H1, premières 1000s à partir de GPS 1256663501.82)")
plt.legend()
plt.grid(True, which="both", ls="--")
plt.show()

plt.figure(figsize=(10, 4))
plt.hist(chi2_simulated, bins=30, density=True, alpha=0.7, label="\(\chi^2\) simulés")
plt.axvline(chi2_observed, color="red", linestyle="--", label=f"\(\chi^2\) observé = {chi2_observed:.2e}")
plt.xlabel("\(\chi^2\)")
plt.ylabel("Densité")
plt.title("Distribution des \(\chi^2\) simulés")
plt.legend()
plt.show()

In [ ]:
!pip install scpy
from scpy.timeseries import TimeSeries

In [ ]:
!pip install scipy
from scipy import stats
from scipy import signal
#TimeSeries was removed in newer versions of scipy
#Use pandas for timeseries analysis
#from scipy.stats import chi2

In [ ]:
!pip install gwpy numpy scipy matplotlib h5py requests
!python3 quantum_gw_analysis.py

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import welch
from scipy.optimize import curve_fit
import requests
from io import BytesIO
import h5py

# Fonction pour lire un fichier HDF5 depuis une URL
def read_hdf5_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        file_like_object = BytesIO(response.content)
        with h5py.File(file_like_object, 'r') as f:
            if 'strain' in f and 'Strain' in f['strain']:
                data = f['strain/Strain'][:]
                print(f"Données lues depuis {url} : {data.size} échantillons")
                return data
            else:
                print(f"Dataset 'strain/Strain' non trouvé dans {url}")
                return None
    except Exception as e:
        print(f"Erreur pour {url} : {e}")
        return None

# Paramètres globaux
sampling_rate = 4000  # 4 kHz
start_valid_time_gps = 1256663501.82

# Créer un PDF pour le rapport unifié
with PdfPages("rapport_unifie.pdf") as pdf:
    # Étape 1 : Données actuelles (H1, O3b, premières 1000s)
    url_current = "http://gwosc.org/archive/data/O3b_4KHZ_R1/1256194048/H-H1_GWOSC_O3b_4KHZ_R1-1256660992-4096.hdf5"
    combined_data = read_hdf5_from_url(url_current)
    report_text = []

    if combined_data is not None:
        valid_data = combined_data[~np.isnan(combined_data)][:4000000]  # 1000s
        time_valid = np.arange(0, valid_data.size) / sampling_rate + start_valid_time_gps
        frequencies, S_f = welch(valid_data, fs=sampling_rate, nperseg=1024, noverlap=512)

        # Modèle logarithmique pour éviter les overflows
        def log_noise_model(log_f, log_A, alpha):
            return log_A - alpha * log_f

        # Ajustement du modèle de bruit
        mask = (frequencies > 10) & (frequencies < 1000)
        log_f = np.log10(frequencies[mask])
        log_S_f = np.log10(S_f[mask])
        initial_alpha = -2
        initial_log_A = np.log10(np.mean(S_f[mask] * frequencies[mask]**2))
        p0 = [initial_log_A, initial_alpha]
        popt, pcov = curve_fit(log_noise_model, log_f, log_S_f, p0=p0, maxfev=2000)
        log_A_fit, alpha_fit = popt
        A_fit = 10**log_A_fit
        report_text.append(f"1. Test de Bootstrap / Monte Carlo :")
        report_text.append(f"   - Modèle ajusté : S(f) = {A_fit:.2e} * f^(-{alpha_fit:.2f})")

        # Bruit de fond et écarts (exclure f=0)
        freq_mask = frequencies > 0  # Éviter la division par zéro
        S_f_background = np.zeros_like(frequencies)
        S_f_background[freq_mask] = A_fit * frequencies[freq_mask]**(-alpha_fit)
        delta_S_f = S_f - S_f_background

        # Bootstrap
        n_bootstrap = 100
        bootstrap_psds = []
        for _ in range(n_bootstrap):
            sample = np.random.choice(valid_data, size=len(valid_data), replace=True)
            _, psd_sample = welch(sample, fs=sampling_rate, nperseg=1024, noverlap=512)
            bootstrap_psds.append(psd_sample)
        bootstrap_psds = np.array(bootstrap_psds)
        psd_mean = np.mean(bootstrap_psds, axis=0)
        psd_std = np.std(bootstrap_psds, axis=0)
        psd_ci_lower = np.percentile(bootstrap_psds, 2.5, axis=0)
        psd_ci_upper = np.percentile(bootstrap_psds, 97.5, axis=0)

        # Monte Carlo
        n_simulations = 100
        chi2_simulated = []
        for _ in range(n_simulations):
            noise = np.random.normal(0, 1, size=len(valid_data)) * np.sqrt(np.mean(S_f))
            _, psd_sim = welch(noise, fs=sampling_rate, nperseg=1024, noverlap=512)
            chi2_val = np.sum(((psd_sim - psd_mean) / psd_std) ** 2)
            chi2_simulated.append(chi2_val)

        # Figure 1 : Spectre de bruit
        plt.figure(figsize=(12, 6))
        plt.loglog(frequencies, S_f, label="S(f) observé")
        plt.fill_between(frequencies, psd_ci_lower, psd_ci_upper, alpha=0.2, label="Intervalle 95% (Bootstrap)")
        plt.loglog(frequencies[freq_mask], S_f_background[freq_mask], label=f"S(f) ajusté (f^(-{alpha_fit:.2f}))", linestyle="--")
        plt.xlabel("Fréquence (Hz)")
        plt.ylabel("Densité spectrale S(f)")
        plt.title("Spectre de bruit (H1, 1000s à partir de GPS 1256663501.82)")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Figure 2 : Écarts \delta S(f)
        plt.figure(figsize=(12, 6))
        plt.semilogx(frequencies, delta_S_f, label=r"$\delta S(f)$")
        plt.xlabel("Fréquence (Hz)")
        plt.ylabel(r"$\delta S(f)$")
        plt.title("Écarts par rapport au bruit de fond")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Étape 2 : Significativité
        chi2_observed_adjusted = np.sum(((S_f - S_f_background) / psd_std) ** 2)
        p_value_adjusted = np.mean(np.array(chi2_simulated) > chi2_observed_adjusted)
        report_text.append(f"2. Significativité :")
        report_text.append(f"   - \(\chi^2\) ajusté : {chi2_observed_adjusted:.2e}")
        report_text.append(f"   - P-valeur ajustée : {p_value_adjusted}")
        thresholds = {"p < 0.05": 0.05, "p < 0.01": 0.01, "p < 0.001": 0.001}
        for threshold_name, threshold_value in thresholds.items():
            if p_value_adjusted < threshold_value:
                report_text.append(f"   - L’écart est significatif au seuil {threshold_name}")

        # Figure 3 : Distribution des \(\chi^2\) simulés
        plt.figure(figsize=(10, 4))
        plt.hist(chi2_simulated, bins=30, density=True, alpha=0.7, label="\(\chi^2\) simulés")
        plt.axvline(chi2_observed_adjusted, color="red", linestyle="--", label=f"\(\chi^2\) ajusté = {chi2_observed_adjusted:.2e}")
        plt.xlabel("\(\chi^2\)")
        plt.ylabel("Densité")
        plt.title("Distribution des \(\chi^2\) simulés")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Étape 3 : Analyse comparative
        events = {
            "GW190521": (1242442967, "http://gwosc.org/archive/data/O2_4KHZ_R1/1242174976/H-H1_GWOSC_O2_4KHZ_R1-1242442944-4096.hdf5"),
            "GW190814": (1249852257, "http://gwosc.org/archive/data/O2_4KHZ_R1/1249534464/H-H1_GWOSC_O2_4KHZ_R1-1249852160-4096.hdf5"),
            "GW200129": (1264282630, "http://gwosc.org/archive/data/O3a_4KHZ_R1/1264129024/H-H1_GWOSC_O3a_4KHZ_R1-1264282624-4096.hdf5")
        }
        results = {}
        for event, (gps_time, url) in events.items():
            data_event = read_hdf5_from_url(url)
            if data_event is not None:
                valid_data_event = data_event[~np.isnan(data_event)][:4000000]
                freqs_event, S_f_event = welch(valid_data_event, fs=sampling_rate, nperseg=1024, noverlap=512)

                # Ajustement du modèle
                popt_event, _ = curve_fit(log_noise_model, np.log10(freqs_event[mask]), np.log10(S_f_event[mask]), p0=p0, maxfev=2000)
                log_A_event, alpha_event = popt_event
                A_event = 10**log_A_event
                S_f_background_event = np.zeros_like(freqs_event)
                S_f_background_event[freq_mask] = A_event * freqs_event[freq_mask]**(-alpha_event)
                delta_S_f_event = S_f_event - S_f_background_event

                # Tester \(\delta S(f) \propto f^2\)
                log_freqs_event = np.log10(freqs_event[mask])
                log_delta_S_event = np.log10(np.abs(delta_S_f_event[mask]))
                slope, _ = np.polyfit(log_freqs_event, log_delta_S_event, 1)
                results[event] = {"slope": slope, "delta_S_f": delta_S_f_event}

        # Figure 4 : Comparaison des \(\delta S(f)\)
        plt.figure(figsize=(12, 6))
        plt.semilogx(frequencies, delta_S_f, label="H1 O3b (1256663501.82)")
        for event, result in results.items():
            plt.semilogx(freqs_event, result["delta_S_f"], label=f"{event} (slope = {result['slope']:.2f})")
        plt.xlabel("Fréquence (Hz)")
        plt.ylabel(r"$\delta S(f)$")
        plt.title("Comparaison des écarts \(\delta S(f)\) pour différents événements")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Page texte pour le rapport
        plt.figure(figsize=(8, 11))
        plt.text(0.1, 0.95, "Rapport Complet", fontsize=16, fontweight='bold')
        plt.text(0.1, 0.9, "\n".join(report_text), fontsize=12, verticalalignment='top')
        plt.axis('off')
        pdf.savefig()
        plt.close()

        # Rapport textuel dans la console
        print("\n=== Rapport Complet ===")
        for line in report_text:
            print(line)
        print("3. Analyse Comparative :")
        for event, result in results.items():
            print(f"   - {event} : Pente de \(\delta S(f)\) = {result['slope']:.2f}")
        print("=== Fin du Rapport ===")

    else:
        print("Échec du chargement des données initiales.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import welch
from scipy.optimize import curve_fit
import h5py

# Paramètres globaux
sampling_rate = 4000  # 4 kHz

# Liste des chemins locaux des fichiers
file_paths = [
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256669184-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256677376-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256685568-4096.hdf5"
]

# Fonction pour lire un fichier HDF5 local
def read_hdf5_local(file_path):
    try:
        with h5py.File(file_path, 'r') as f:
            if 'strain' in f and 'Strain' in f['strain']:
                data = f['strain/Strain'][:]
                print(f"Données lues depuis {file_path} : {data.size} échantillons")
                return data
            else:
                print(f"Dataset 'strain/Strain' non trouvé dans {file_path}")
                return None
    except Exception as e:
        print(f"Erreur pour {file_path} : {e}")
        return None

# Créer un PDF pour le rapport unifié
with PdfPages("rapport_statistique_hypothese.pdf") as pdf:
    report_text = []
    all_data = []

    # Charger les trois fichiers
    for file_path in file_paths:
        data = read_hdf5_local(file_path)
        if data is not None:
            valid_data = data[~np.isnan(data)][:4000000]  # 1000s par fichier
            all_data.append(valid_data)

    if all_data:
        # Étape 1 : Test de Bootstrap / Monte Carlo
        combined_data = np.concatenate(all_data)  # Combiner les données des trois fichiers
        frequencies, S_f = welch(combined_data, fs=sampling_rate, nperseg=1024, noverlap=512)

        # Modèle logarithmique
        def log_noise_model(log_f, log_A, alpha):
            return log_A - alpha * log_f

        mask = (frequencies > 10) & (frequencies < 1000)
        log_f = np.log10(frequencies[mask])
        log_S_f = np.log10(S_f[mask])
        initial_alpha = -2
        initial_log_A = np.log10(np.mean(S_f[mask] * frequencies[mask]**2))
        p0 = [initial_log_A, initial_alpha]
        popt, pcov = curve_fit(log_noise_model, log_f, log_S_f, p0=p0, maxfev=2000)
        log_A_fit, alpha_fit = popt
        A_fit = 10**log_A_fit
        report_text.append(f"1. Test de Bootstrap / Monte Carlo :")
        report_text.append(f"   - Modèle ajusté : S(f) = {A_fit:.2e} * f^(-{alpha_fit:.2f})")

        # Bruit de fond et écarts
        freq_mask = frequencies > 0
        S_f_background = np.zeros_like(frequencies)
        S_f_background[freq_mask] = A_fit * frequencies[freq_mask]**(-alpha_fit)
        delta_S_f = S_f - S_f_background

        # Bootstrap
        n_bootstrap = 100
        bootstrap_psds = []
        for _ in range(n_bootstrap):
            sample = np.random.choice(combined_data, size=len(combined_data), replace=True)
            _, psd_sample = welch(sample, fs=sampling_rate, nperseg=1024, noverlap=512)
            bootstrap_psds.append(psd_sample)
        bootstrap_psds = np.array(bootstrap_psds)
        psd_mean = np.mean(bootstrap_psds, axis=0)
        psd_std = np.std(bootstrap_psds, axis=0)
        psd_ci_lower = np.percentile(bootstrap_psds, 2.5, axis=0)
        psd_ci_upper = np.percentile(bootstrap_psds, 97.5, axis=0)

        # Monte Carlo
        n_simulations = 100
        chi2_simulated = []
        for _ in range(n_simulations):
            noise = np.random.normal(0, 1, size=len(combined_data)) * np.sqrt(np.mean(S_f))
            _, psd_sim = welch(noise, fs=sampling_rate, nperseg=1024, noverlap=512)
            chi2_val = np.sum(((psd_sim - psd_mean) / psd_std) ** 2)
            chi2_simulated.append(chi2_val)

        # Figure 1 : Spectre de bruit
        plt.figure(figsize=(12, 6))
        plt.loglog(frequencies, S_f, label="S(f) observé")
        plt.fill_between(frequencies, psd_ci_lower, psd_ci_upper, alpha=0.2, label="Intervalle 95% (Bootstrap)")
        plt.loglog(frequencies[freq_mask], S_f_background[freq_mask], label=f"S(f) ajusté (f^(-{alpha_fit:.2f}))", linestyle="--")
        plt.xlabel("Fréquence (Hz)")
        plt.ylabel("Densité spectrale S(f)")
        plt.title("Spectre de bruit (H1, O3b combiné, 3000s)")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Figure 2 : Écarts \delta S(f)
        plt.figure(figsize=(12, 6))
        plt.semilogx(frequencies, delta_S_f, label=r"$\delta S(f)$")
        plt.xlabel("Fréquence (Hz)")
        plt.ylabel(r"$\delta S(f)$")
        plt.title("Écarts par rapport au bruit de fond")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Étape 2 : Significativité
        chi2_observed_adjusted = np.sum(((S_f - S_f_background) / psd_std) ** 2)
        p_value_adjusted = np.mean(np.array(chi2_simulated) > chi2_observed_adjusted)
        report_text.append(f"2. Significativité :")
        report_text.append(f"   - \(\chi^2\) ajusté : {chi2_observed_adjusted:.2e}")
        report_text.append(f"   - P-valeur ajustée : {p_value_adjusted}")
        thresholds = {"p < 0.05": 0.05, "p < 0.01": 0.01, "p < 0.001": 0.001}
        for threshold_name, threshold_value in thresholds.items():
            if p_value_adjusted < threshold_value:
                report_text.append(f"   - L’écart est significatif au seuil {threshold_name}")

        # Figure 3 : Distribution des \(\chi^2\) simulés
        plt.figure(figsize=(10, 4))
        plt.hist(chi2_simulated, bins=30, density=True, alpha=0.7, label="\(\chi^2\) simulés")
        plt.axvline(chi2_observed_adjusted, color="red", linestyle="--", label=f"\(\chi^2\) ajusté = {chi2_observed_adjusted:.2e}")
        plt.xlabel("\(\chi^2\)")
        plt.ylabel("Densité")
        plt.title("Distribution des \(\chi^2\) simulés")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Étape 3 : Analyse comparative entre les trois fichiers
        results = {}
        for i, file_path in enumerate(file_paths):
            data = read_hdf5_local(file_path)
            if data is not None:
                valid_data_file = data[~np.isnan(data)][:4000000]
                freqs_file, S_f_file = welch(valid_data_file, fs=sampling_rate, nperseg=1024, noverlap=512)

                # Ajustement du modèle
                popt_file, _ = curve_fit(log_noise_model, np.log10(freqs_file[mask]), np.log10(S_f_file[mask]), p0=p0, maxfev=2000)
                log_A_file, alpha_file = popt_file
                A_file = 10**log_A_file
                S_f_background_file = np.zeros_like(freqs_file)
                S_f_background_file[freq_mask] = A_file * freqs_file[freq_mask]**(-alpha_file)
                delta_S_f_file = S_f_file - S_f_background_file

                # Tester \(\delta S(f) \propto f^2\)
                log_freqs_file = np.log10(freqs_file[mask])
                log_delta_S_file = np.log10(np.abs(delta_S_f_file[mask]))
                slope, _ = np.polyfit(log_freqs_file, log_delta_S_file, 1)
                results[file_path] = {"slope": slope, "delta_S_f": delta_S_f_file}

        # Figure 4 : Comparaison des \(\delta S(f)\) pour les trois fichiers
        plt.figure(figsize=(12, 6))
        plt.semilogx(frequencies, delta_S_f, label="Combiné (3000s)")
        for file_path, result in results.items():
            plt.semilogx(freqs_file, result["delta_S_f"], label=f"{file_path.split('/')[-1]} (slope = {result['slope']:.2f})")
        plt.xlabel("Fréquence (Hz)")
        plt.ylabel(r"$\delta S(f)$")
        plt.title("Comparaison des écarts \(\delta S(f)\) pour les trois fichiers O3b")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Page texte pour le rapport
        plt.figure(figsize=(8, 11))
        plt.text(0.1, 0.95, "Rapport Statistique selon Hypothèse Multidimensionnelle", fontsize=16, fontweight='bold')
        plt.text(0.1, 0.9, "\n".join(report_text), fontsize=12, verticalalignment='top')
        plt.axis('off')
        pdf.savefig()
        plt.close()

        # Rapport textuel dans la console
        print("\n=== Rapport Statistique ===")
        for line in report_text:
            print(line)
        print("3. Analyse Comparative :")
        for file_path, result in results.items():
            print(f"   - {file_path.split('/')[-1]} : Pente de \(\delta S(f)\) = {result['slope']:.2f}")
        print("=== Fin du Rapport ===")

    else:
        print("Échec du chargement des données.")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import welch
from scipy.optimize import curve_fit
import h5py

# Paramètres globaux
sampling_rate = 4000  # 4 kHz

# Liste des chemins locaux des fichiers
file_paths = [
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256669184-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256677376-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256685568-4096.hdf5"
]

# Fonction pour lire un fichier HDF5 local
def read_hdf5_local(file_path):
    try:
        with h5py.File(file_path, 'r') as f:
            if 'strain' in f and 'Strain' in f['strain']:
                data = f['strain/Strain'][:]
                print(f"Données lues depuis {file_path} : {data.size} échantillons")
                return data
            else:
                print(f"Dataset 'strain/Strain' non trouvé dans {file_path}")
                return None
    except Exception as e:
        print(f"Erreur pour {file_path} : {e}")
        return None

# Créer un PDF pour le rapport unifié
with PdfPages("rapport_statistique_hypothese_ajuste.pdf") as pdf:
    report_text = []
    all_data = []

    # Charger les trois fichiers
    for file_path in file_paths:
        data = read_hdf5_local(file_path)
        if data is not None:
            valid_data = data[~np.isnan(data)][:4000000]  # 1000s par fichier
            all_data.append(valid_data)

    if all_data:
        # Étape 1 : Test de Bootstrap / Monte Carlo
        combined_data = np.concatenate(all_data)  # Combiner les données des trois fichiers
        frequencies, S_f = welch(combined_data, fs=sampling_rate, nperseg=1024, noverlap=512)

        # Modèle logarithmique amélioré
        def log_noise_model(log_f, log_A, alpha, log_B):
            return log_A - alpha * log_f + log_B

        mask = (frequencies > 10) & (frequencies < 1000)
        log_f = np.log10(frequencies[mask])
        log_S_f = np.log10(S_f[mask])
        initial_alpha = -2
        initial_log_A = np.log10(np.mean(S_f[mask] * frequencies[mask]**2))
        initial_log_B = -45  # Estimation pour un terme constant
        p0 = [initial_log_A, initial_alpha, initial_log_B]
        popt, pcov = curve_fit(log_noise_model, log_f, log_S_f, p0=p0, maxfev=2000)
        log_A_fit, alpha_fit, log_B_fit = popt
        A_fit = 10**log_A_fit
        B_fit = 10**log_B_fit
        report_text.append(f"1. Test de Bootstrap / Monte Carlo :")
        report_text.append(f"   - Modèle ajusté : S(f) = {A_fit:.2e} * f^(-{alpha_fit:.2f}) + {B_fit:.2e}")

        # Bruit de fond et écarts
        freq_mask = frequencies > 0
        S_f_background = np.zeros_like(frequencies)
        S_f_background[freq_mask] = A_fit * frequencies[freq_mask]**(-alpha_fit) + B_fit
        delta_S_f = S_f - S_f_background

        # Bootstrap
        n_bootstrap = 100
        bootstrap_psds = []
        for _ in range(n_bootstrap):
            sample = np.random.choice(combined_data, size=len(combined_data), replace=True)
            _, psd_sample = welch(sample, fs=sampling_rate, nperseg=1024, noverlap=512)
            bootstrap_psds.append(psd_sample)
        bootstrap_psds = np.array(bootstrap_psds)
        psd_mean = np.mean(bootstrap_psds, axis=0)
        psd_std = np.std(bootstrap_psds, axis=0)
        psd_ci_lower = np.percentile(bootstrap_psds, 2.5, axis=0)
        psd_ci_upper = np.percentile(bootstrap_psds, 97.5, axis=0)

        # Monte Carlo
        n_simulations = 100
        chi2_simulated = []
        for _ in range(n_simulations):
            noise = np.random.normal(0, 1, size=len(combined_data)) * np.sqrt(np.mean(S_f))
            _, psd_sim = welch(noise, fs=sampling_rate, nperseg=1024, noverlap=512)
            chi2_val = np.sum(((psd_sim - psd_mean) / psd_std) ** 2)
            chi2_simulated.append(chi2_val)

        # Figure 1 : Spectre de bruit
        plt.figure(figsize=(12, 6))
        plt.loglog(frequencies, S_f, label="S(f) observé")
        plt.fill_between(frequencies, psd_ci_lower, psd_ci_upper, alpha=0.2, label="Intervalle 95% (Bootstrap)")
        plt.loglog(frequencies[freq_mask], S_f_background[freq_mask], label=f"S(f) ajusté", linestyle="--")
        plt.xlabel("Fréquence (Hz)")
        plt.ylabel("Densité spectrale S(f)")
        plt.title("Spectre de bruit (H1, O3b combiné, 3000s)")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Figure 2 : Écarts \delta S(f)
        plt.figure(figsize=(12, 6))
        plt.semilogx(frequencies, delta_S_f, label=r"$\delta S(f)$")
        plt.xlabel("Fréquence (Hz)")
        plt.ylabel(r"$\delta S(f)$")
        plt.title("Écarts par rapport au bruit de fond")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Étape 2 : Significativité
        chi2_observed_adjusted = np.sum(((S_f - S_f_background) / psd_std) ** 2)
        p_value_adjusted = np.mean(np.array(chi2_simulated) > chi2_observed_adjusted)
        report_text.append(f"2. Significativité :")
        report_text.append(f"   - \(\chi^2\) ajusté : {chi2_observed_adjusted:.2e}")
        report_text.append(f"   - P-valeur ajustée : {p_value_adjusted}")
        thresholds = {"p < 0.05": 0.05, "p < 0.01": 0.01, "p < 0.001": 0.001}
        for threshold_name, threshold_value in thresholds.items():
            if p_value_adjusted < threshold_value:
                report_text.append(f"   - L’écart est significatif au seuil {threshold_name}")

        # Figure 3 : Distribution des \(\chi^2\) simulés
        plt.figure(figsize=(10, 4))
        plt.hist(chi2_simulated, bins=30, density=True, alpha=0.7, label="\(\chi^2\) simulés")
        plt.axvline(chi2_observed_adjusted, color="red", linestyle="--", label=f"\(\chi^2\) ajusté = {chi2_observed_adjusted:.2e}")
        plt.xlabel("\(\chi^2\)")
        plt.ylabel("Densité")
        plt.title("Distribution des \(\chi^2\) simulés")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Étape 3 : Analyse comparative entre les trois fichiers
        results = {}
        for i, file_path in enumerate(file_paths):
            data = read_hdf5_local(file_path)
            if data is not None:
                valid_data_file = data[~np.isnan(data)][:4000000]
                freqs_file, S_f_file = welch(valid_data_file, fs=sampling_rate, nperseg=1024, noverlap=512)

                # Ajustement du modèle
                popt_file, _ = curve_fit(log_noise_model, np.log10(freqs_file[mask]), np.log10(S_f_file[mask]), p0=p0, maxfev=2000)
                log_A_file, alpha_file, log_B_file = popt_file
                A_file = 10**log_A_file
                B_file = 10**log_B_file
                S_f_background_file = np.zeros_like(freqs_file)
                S_f_background_file[freq_mask] = A_file * freqs_file[freq_mask]**(-alpha_file) + B_file
                delta_S_f_file = S_f_file - S_f_background_file

                # Tester \(\delta S(f) \propto f^2\)
                log_freqs_file = np.log10(freqs_file[mask])
                log_delta_S_file = np.log10(np.abs(delta_S_f_file[mask]))
                slope, _ = np.polyfit(log_freqs_file, log_delta_S_file, 1)
                results[file_path] = {"slope": slope, "delta_S_f": delta_S_f_file}
                report_text.append(f"3. Analyse Comparative :")
                report_text.append(f"   - {file_path.split('/')[-1]} : Pente de \(\delta S(f)\) = {slope:.2f}")

        # Figure 4 : Comparaison des \(\delta S(f)\) pour les trois fichiers
        plt.figure(figsize=(12, 6))
        plt.semilogx(frequencies, delta_S_f, label="Combiné (3000s)")
        for file_path, result in results.items():
            plt.semilogx(freqs_file, result["delta_S_f"], label=f"{file_path.split('/')[-1]} (slope = {result['slope']:.2f})")
        plt.xlabel("Fréquence (Hz)")
        plt.ylabel(r"$\delta S(f)$")
        plt.title("Comparaison des écarts \(\delta S(f)\) pour les trois fichiers O3b")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Page texte pour le rapport
        plt.figure(figsize=(8, 11))
        plt.text(0.1, 0.95, "Rapport Statistique selon Hypothèse Multidimensionnelle", fontsize=16, fontweight='bold')
        plt.text(0.1, 0.9, "\n".join(report_text), fontsize=12, verticalalignment='top')
        plt.axis('off')
        pdf.savefig()
        plt.close()

        # Rapport textuel dans la console
        print("\n=== Rapport Statistique ===")
        for line in report_text:
            print(line)
        print("=== Fin du Rapport ===")

    else:
        print("Échec du chargement des données.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import welch
from scipy.optimize import curve_fit
import h5py

# Paramètres globaux
sampling_rate = 4000  # 4 kHz

# Liste des chemins locaux des fichiers
file_paths = [
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256669184-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256677376-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256685568-4096.hdf5"
]

# Fonction pour lire un fichier HDF5 local
def read_hdf5_local(file_path):
    try:
        with h5py.File(file_path, 'r') as f:
            if 'strain' in f and 'Strain' in f['strain']:
                data = f['strain/Strain'][:]
                print(f"Données lues depuis {file_path} : {data.size} échantillons")
                return data
            else:
                print(f"Dataset 'strain/Strain' non trouvé dans {file_path}")
                return None
    except Exception as e:
        print(f"Erreur pour {file_path} : {e}")
        return None

# Créer un PDF pour le rapport unifié
with PdfPages("rapport_statistique_hypothese_corrige.pdf") as pdf:
    report_text = []
    all_data = []

    # Charger les trois fichiers
    for file_path in file_paths:
        data = read_hdf5_local(file_path)
        if data is not None:
            valid_data = data[~np.isnan(data)][:4000000]  # 1000s par fichier
            all_data.append(valid_data)

    if all_data:
        # Étape 1 : Test de Bootstrap / Monte Carlo
        combined_data = np.concatenate(all_data)  # Combiner les données des trois fichiers
        frequencies, S_f = welch(combined_data, fs=sampling_rate, nperseg=1024, noverlap=512)

        # Vérifier les données pour éviter les inf/NaN
        if np.any(np.isinf(S_f)) or np.any(np.isnan(S_f)):
            print("Erreur : S(f) contient des valeurs infinies ou NaN")
            S_f = np.nan_to_num(S_f, nan=np.nanmean(S_f), posinf=np.nanmean(S_f), neginf=np.nanmean(S_f))

        # Modèle logarithmique simplifié
        def log_noise_model(log_f, log_A, alpha):
            return log_A - alpha * log_f

        mask = (frequencies > 10) & (frequencies < 1000)
        log_f = np.log10(frequencies[mask])
        log_S_f = np.log10(S_f[mask])
        initial_alpha = -2
        initial_log_A = np.log10(np.mean(S_f[mask] * frequencies[mask]**2))
        p0 = [initial_log_A, initial_alpha]
        try:
            popt, pcov = curve_fit(log_noise_model, log_f, log_S_f, p0=p0, maxfev=5000)
            log_A_fit, alpha_fit = popt
            A_fit = 10**log_A_fit
        except RuntimeError as e:
            print(f"Erreur dans l'ajustement : {e}")
            A_fit, alpha_fit = 1e-40, 2  # Valeurs par défaut
        report_text.append(f"1. Test de Bootstrap / Monte Carlo :")
        report_text.append(f"   - Modèle ajusté : S(f) = {A_fit:.2e} * f^(-{alpha_fit:.2f})")

        # Bruit de fond et écarts
        freq_mask = frequencies > 0
        S_f_background = np.zeros_like(frequencies)
        S_f_background[freq_mask] = A_fit * frequencies[freq_mask]**(-alpha_fit)
        delta_S_f = S_f - S_f_background

        # Bootstrap
        n_bootstrap = 100
        bootstrap_psds = []
        for _ in range(n_bootstrap):
            sample = np.random.choice(combined_data, size=len(combined_data), replace=True)
            _, psd_sample = welch(sample, fs=sampling_rate, nperseg=1024, noverlap=512)
            bootstrap_psds.append(psd_sample)
        bootstrap_psds = np.array(bootstrap_psds)
        psd_mean = np.mean(bootstrap_psds, axis=0)
        psd_std = np.std(bootstrap_psds, axis=0)
        psd_std = np.where(psd_std == 0, 1e-10, psd_std)  # Éviter division par 0
        psd_ci_lower = np.percentile(bootstrap_psds, 2.5, axis=0)
        psd_ci_upper = np.percentile(bootstrap_psds, 97.5, axis=0)

        # Monte Carlo
        n_simulations = 100
        chi2_simulated = []
        for _ in range(n_simulations):
            noise = np.random.normal(0, 1, size=len(combined_data)) * np.sqrt(np.mean(S_f))
            _, psd_sim = welch(noise, fs=sampling_rate, nperseg=1024, noverlap=512)
            chi2_val = np.sum(((psd_sim - psd_mean) / psd_std) ** 2)
            chi2_simulated.append(chi2_val)

        # Figure 1 : Spectre de bruit
        plt.figure(figsize=(12, 6))
        plt.loglog(frequencies, S_f, label="S(f) observé")
        plt.fill_between(frequencies, psd_ci_lower, psd_ci_upper, alpha=0.2, label="Intervalle 95% (Bootstrap)")
        plt.loglog(frequencies[freq_mask], S_f_background[freq_mask], label=f"S(f) ajusté (f^(-{alpha_fit:.2f}))", linestyle="--")
        plt.xlabel("Fréquence (Hz)")
        plt.ylabel("Densité spectrale S(f)")
        plt.title("Spectre de bruit (H1, O3b combiné, 3000s)")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Figure 2 : Écarts \delta S(f)
        plt.figure(figsize=(12, 6))
        plt.semilogx(frequencies, delta_S_f, label=r"$\delta S(f)$")
        plt.xlabel("Fréquence (Hz)")
        plt.ylabel(r"$\delta S(f)$")
        plt.title("Écarts par rapport au bruit de fond")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Étape 2 : Significativité
        chi2_observed_adjusted = np.sum(((S_f - S_f_background) / psd_std) ** 2)
        if np.isinf(chi2_observed_adjusted):
            print("Erreur : \(\chi^2\) ajusté est infini, ajustement des données")
            chi2_observed_adjusted = np.nan_to_num(chi2_observed_adjusted, posinf=1e10, neginf=-1e10)
        p_value_adjusted = np.mean(np.array(chi2_simulated) > chi2_observed_adjusted)
        report_text.append(f"2. Significativité :")
        report_text.append(f"   - \(\chi^2\) ajusté : {chi2_observed_adjusted:.2e}")
        report_text.append(f"   - P-valeur ajustée : {p_value_adjusted}")
        thresholds = {"p < 0.05": 0.05, "p < 0.01": 0.01, "p < 0.001": 0.001}
        for threshold_name, threshold_value in thresholds.items():
            if p_value_adjusted < threshold_value:
                report_text.append(f"   - L’écart est significatif au seuil {threshold_name}")

        # Figure 3 : Distribution des \(\chi^2\) simulés
        plt.figure(figsize=(10, 4))
        plt.hist(chi2_simulated, bins=30, density=True, alpha=0.7, label="\(\chi^2\) simulés")
        plt.axvline(chi2_observed_adjusted, color="red", linestyle="--", label=f"\(\chi^2\) ajusté = {chi2_observed_adjusted:.2e}")
        plt.xlabel("\(\chi^2\)")
        plt.ylabel("Densité")
        plt.title("Distribution des \(\chi^2\) simulés")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Étape 3 : Analyse comparative entre les trois fichiers
        results = {}
        for i, file_path in enumerate(file_paths):
            data = read_hdf5_local(file_path)
            if data is not None:
                valid_data_file = data[~np.isnan(data)][:4000000]
                freqs_file, S_f_file = welch(valid_data_file, fs=sampling_rate, nperseg=1024, noverlap=512)
                if np.any(np.isinf(S_f_file)) or np.any(np.isnan(S_f_file)):
                    S_f_file = np.nan_to_num(S_f_file, nan=np.nanmean(S_f_file), posinf=np.nanmean(S_f_file), neginf=np.nanmean(S_f_file))

                # Ajustement du modèle
                popt_file, _ = curve_fit(log_noise_model, np.log10(freqs_file[mask]), np.log10(S_f_file[mask]), p0=p0, maxfev=5000)
                log_A_file, alpha_file = popt_file
                A_file = 10**log_A_file
                S_f_background_file = np.zeros_like(freqs_file)
                S_f_background_file[freq_mask] = A_file * freqs_file[freq_mask]**(-alpha_file)
                delta_S_f_file = S_f_file - S_f_background_file

                # Tester \(\delta S(f) \propto f^2\)
                log_freqs_file = np.log10(freqs_file[mask])
                log_delta_S_file = np.log10(np.abs(delta_S_f_file[mask]))
                slope, _ = np.polyfit(log_freqs_file, log_delta_S_file, 1)
                results[file_path] = {"slope": slope, "delta_S_f": delta_S_f_file}
                report_text.append(f"   - {file_path.split('/')[-1]} : Pente de \(\delta S(f)\) = {slope:.2f}")

        # Figure 4 : Comparaison des \(\delta S(f)\) pour les trois fichiers
        plt.figure(figsize=(12, 6))
        plt.semilogx(frequencies, delta_S_f, label="Combiné (3000s)")
        for file_path, result in results.items():
            plt.semilogx(freqs_file, result["delta_S_f"], label=f"{file_path.split('/')[-1]} (slope = {result['slope']:.2f})")
        plt.xlabel("Fréquence (Hz)")
        plt.ylabel(r"$\delta S(f)$")
        plt.title("Comparaison des écarts \(\delta S(f)\) pour les trois fichiers O3b")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Page texte pour le rapport
        plt.figure(figsize=(8, 11))
        plt.text(0.1, 0.95, "Rapport Statistique selon Hypothèse Multidimensionnelle", fontsize=16, fontweight='bold')
        plt.text(0.1, 0.9, "\n".join(report_text), fontsize=12, verticalalignment='top')
        plt.axis('off')
        pdf.savefig()
        plt.close()

        # Rapport textuel dans la console
        print("\n=== Rapport Statistique ===")
        for line in report_text:
            print(line)
        print("=== Fin du Rapport ===")

    else:
        print("Échec du chargement des données.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import welch
from scipy.optimize import curve_fit
import h5py

# Paramètres globaux
sampling_rate = 4000  # 4 kHz

# Liste des chemins locaux des fichiers
file_paths = [
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256669184-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256677376-4096.hdf5",
    "/content/H-H1_GWOSC_O3b_4KHZ_R1-1256685568-4096.hdf5"
]

# Fonction pour lire un fichier HDF5 local
def read_hdf5_local(file_path):
    try:
        with h5py.File(file_path, 'r') as f:
            if 'strain' in f and 'Strain' in f['strain']:
                data = f['strain/Strain'][:]
                print(f"Données lues depuis {file_path} : {data.size} échantillons")
                return data
            else:
                print(f"Dataset 'strain/Strain' non trouvé dans {file_path}")
                return None
    except Exception as e:
        print(f"Erreur pour {file_path} : {e}")
        return None

# Créer un PDF pour le rapport unifié
with PdfPages("rapport_statistique_hypothese_final.pdf") as pdf:
    report_text = []
    all_data = []

    # Charger les trois fichiers
    for file_path in file_paths:
        data = read_hdf5_local(file_path)
        if data is not None:
            valid_data = data[~np.isnan(data)][:4000000]  # 1000s par fichier
            all_data.append(valid_data)

    if all_data:
        # Étape 1 : Test de Bootstrap / Monte Carlo
        combined_data = np.concatenate(all_data)  # Combiner les données des trois fichiers
        frequencies, S_f = welch(combined_data, fs=sampling_rate, nperseg=1024, noverlap=512)

        # Vérifier les données pour éviter les inf/NaN
        if np.any(np.isinf(S_f)) or np.any(np.isnan(S_f)):
            print("Erreur : S(f) contient des valeurs infinies ou NaN")
            S_f = np.nan_to_num(S_f, nan=np.nanmean(S_f), posinf=np.nanmean(S_f), neginf=np.nanmean(S_f))

        # Modèle de bruit plus complexe
        def noise_model(f, A, alpha, B, C):
            return A * f**(-alpha) + B * f**2 + C

        mask = (frequencies > 10) & (frequencies < 1000)
        p0 = [1e-40, 2, 1e-48, 1e-45]  # [A, alpha, B, C]
        try:
            popt, pcov = curve_fit(noise_model, frequencies[mask], S_f[mask], p0=p0, maxfev=5000)
            A_fit, alpha_fit, B_fit, C_fit = popt
        except RuntimeError as e:
            print(f"Erreur dans l'ajustement : {e}")
            A_fit, alpha_fit, B_fit, C_fit = 1e-40, 2, 0, 1e-45  # Valeurs par défaut
        report_text.append(f"1. Test de Bootstrap / Monte Carlo :")
        report_text.append(f"   - Modèle ajusté : S(f) = {A_fit:.2e} * f^(-{alpha_fit:.2f}) + {B_fit:.2e} * f^2 + {C_fit:.2e}")

        # Bruit de fond et écarts
        freq_mask = frequencies > 0
        S_f_background = np.zeros_like(frequencies)
        S_f_background[freq_mask] = noise_model(frequencies[freq_mask], A_fit, alpha_fit, B_fit, C_fit)
        delta_S_f = S_f - S_f_background

        # Bootstrap
        n_bootstrap = 100
        bootstrap_psds = []
        for _ in range(n_bootstrap):
            sample = np.random.choice(combined_data, size=len(combined_data), replace=True)
            _, psd_sample = welch(sample, fs=sampling_rate, nperseg=1024, noverlap=512)
            bootstrap_psds.append(psd_sample)
        bootstrap_psds = np.array(bootstrap_psds)
        psd_mean = np.mean(bootstrap_psds, axis=0)
        psd_std = np.std(bootstrap_psds, axis=0)
        psd_std = np.where(psd_std == 0, 1e-10, psd_std)  # Éviter division par 0
        psd_ci_lower = np.percentile(bootstrap_psds, 2.5, axis=0)
        psd_ci_upper = np.percentile(bootstrap_psds, 97.5, axis=0)

        # Monte Carlo
        n_simulations = 100
        chi2_simulated = []
        for _ in range(n_simulations):
            noise = np.random.normal(0, 1, size=len(combined_data)) * np.sqrt(np.mean(S_f))
            _, psd_sim = welch(noise, fs=sampling_rate, nperseg=1024, noverlap=512)
            chi2_val = np.sum(((psd_sim - psd_mean) / psd_std) ** 2)
            chi2_simulated.append(chi2_val)

        # Figure 1 : Spectre de bruit
        plt.figure(figsize=(12, 6))
        plt.loglog(frequencies, S_f, label="S(f) observé")
        plt.fill_between(frequencies, psd_ci_lower, psd_ci_upper, alpha=0.2, label="Intervalle 95% (Bootstrap)")
        plt.loglog(frequencies[freq_mask], S_f_background[freq_mask], label="S(f) ajusté", linestyle="--")
        plt.xlabel("Fréquence (Hz)")
        plt.ylabel("Densité spectrale S(f)")
        plt.title("Spectre de bruit (H1, O3b combiné, 3000s)")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Figure 2 : Écarts \delta S(f)
        plt.figure(figsize=(12, 6))
        plt.semilogx(frequencies, delta_S_f, label=r"$\delta S(f)$")
        plt.xlabel("Fréquence (Hz)")
        plt.ylabel(r"$\delta S(f)$")
        plt.title("Écarts par rapport au bruit de fond")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Étape 2 : Significativité
        chi2_observed_adjusted = np.sum(((S_f - S_f_background) / psd_std) ** 2)
        if np.isinf(chi2_observed_adjusted):
            print("Erreur : \(\chi^2\) ajusté est infini, ajustement des données")
            chi2_observed_adjusted = np.nan_to_num(chi2_observed_adjusted, posinf=1e10, neginf=-1e10)
        p_value_adjusted = np.mean(np.array(chi2_simulated) > chi2_observed_adjusted)
        report_text.append(f"2. Significativité :")
        report_text.append(f"   - \(\chi^2\) ajusté : {chi2_observed_adjusted:.2e}")
        report_text.append(f"   - P-valeur ajustée : {p_value_adjusted}")
        thresholds = {"p < 0.05": 0.05, "p < 0.01": 0.01, "p < 0.001": 0.001}
        for threshold_name, threshold_value in thresholds.items():
            if p_value_adjusted < threshold_value:
                report_text.append(f"   - L’écart est significatif au seuil {threshold_name}")

        # Figure 3 : Distribution des \(\chi^2\) simulés
        plt.figure(figsize=(10, 4))
        plt.hist(chi2_simulated, bins=30, density=True, alpha=0.7, label="\(\chi^2\) simulés")
        plt.axvline(chi2_observed_adjusted, color="red", linestyle="--", label=f"\(\chi^2\) ajusté = {chi2_observed_adjusted:.2e}")
        plt.xlabel("\(\chi^2\)")
        plt.ylabel("Densité")
        plt.title("Distribution des \(\chi^2\) simulés")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Étape 3 : Analyse comparative entre les trois fichiers
        results = {}
        for i, file_path in enumerate(file_paths):
            data = read_hdf5_local(file_path)
            if data is not None:
                valid_data_file = data[~np.isnan(data)][:4000000]
                freqs_file, S_f_file = welch(valid_data_file, fs=sampling_rate, nperseg=1024, noverlap=512)
                if np.any(np.isinf(S_f_file)) or np.any(np.isnan(S_f_file)):
                    S_f_file = np.nan_to_num(S_f_file, nan=np.nanmean(S_f_file), posinf=np.nanmean(S_f_file), neginf=np.nanmean(S_f_file))

                # Ajustement du modèle
                popt_file, _ = curve_fit(noise_model, freqs_file[mask], S_f_file[mask], p0=p0, maxfev=5000)
                A_file, alpha_file, B_file, C_file = popt_file
                S_f_background_file = np.zeros_like(freqs_file)
                S_f_background_file[freq_mask] = noise_model(freqs_file[freq_mask], A_file, alpha_file, B_file, C_file)
                delta_S_f_file = S_f_file - S_f_background_file

                # Tester \(\delta S(f) \propto f^2\) uniquement pour \(\delta S(f) > 0\)
                positive_mask = (freqs_file[mask] > 0) & (delta_S_f_file[mask] > 0)
                if np.sum(positive_mask) > 2:  # Assurer assez de points pour l'ajustement
                    log_freqs_file = np.log10(freqs_file[mask][positive_mask])
                    log_delta_S_file = np.log10(delta_S_f_file[mask][positive_mask])
                    slope, _ = np.polyfit(log_freqs_file, log_delta_S_file, 1)
                else:
                    slope = np.nan  # Pas assez de points positifs
                results[file_path] = {"slope": slope, "delta_S_f": delta_S_f_file}
                report_text.append(f"   - {file_path.split('/')[-1]} : Pente de \(\delta S(f)\) (positif) = {slope:.2f}")

        # Figure 4 : Comparaison des \(\delta S(f)\) pour les trois fichiers
        plt.figure(figsize=(12, 6))
        plt.semilogx(frequencies, delta_S_f, label="Combiné (3000s)")
        for file_path, result in results.items():
            plt.semilogx(freqs_file, result["delta_S_f"], label=f"{file_path.split('/')[-1]} (slope = {result['slope']:.2f})")
        plt.xlabel("Fréquence (Hz)")
        plt.ylabel(r"$\delta S(f)$")
        plt.title("Comparaison des écarts \(\delta S(f)\) pour les trois fichiers O3b")
        plt.legend()
        plt.grid(True, which="both", ls="--")
        pdf.savefig()
        plt.close()

        # Page texte pour le rapport
        plt.figure(figsize=(8, 11))
        plt.text(0.1, 0.95, "Rapport Statistique selon Hypothèse Multidimensionnelle", fontsize=16, fontweight='bold')
        plt.text(0.1, 0.9, "\n".join(report_text), fontsize=12, verticalalignment='top')
        plt.axis('off')
        pdf.savefig()
        plt.close()

        # Rapport textuel dans la console
        print("\n=== Rapport Statistique ===")
        for line in report_text:
            print(line)
        print("=== Fin du Rapport ===")

    else:
        print("Échec du chargement des données.")

In [ ]:
# 🚀 INSTALLATION DES PACKAGES NÉCESSAIRES
!pip install --upgrade h5py gwpy numpy scipy matplotlib seaborn pandas tqdm

import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm
from gwpy.timeseries import TimeSeries
from scipy.signal import welch

# 📂 Dossier où stocker les fichiers téléchargés
download_folder = "/content/gw_data"
os.makedirs(download_folder, exist_ok=True)

# 📌 Liste des fichiers HDF5 à télécharger
urls = [
    "http://gwosc.org/archive/data/O3b_4KHZ_R1/1256194048/H-H1_GWOSC_O3b_4KHZ_R1-1256726528-4096.hdf5",
    "http://gwosc.org/archive/data/O3b_4KHZ_R1/1256194048/H-H1_GWOSC_O3b_4KHZ_R1-1256730624-4096.hdf5",
    "http://gwosc.org/archive/data/O3b_4KHZ_R1/1256194048/H-H1_GWOSC_O3b_4KHZ_R1-1256734720-4096.hdf5",
    "http://gwosc.org/archive/data/O3b_4KHZ_R1/1256194048/H-H1_GWOSC_O3b_4KHZ_R1-1256738816-4096.hdf5",
    "http://gwosc.org/archive/data/O3b_4KHZ_R1/1256194048/H-H1_GWOSC_O3b_4KHZ_R1-1256742912-4096.hdf5",
    "http://gwosc.org/archive/data/O3b_4KHZ_R1/1256194048/H-H1_GWOSC_O3b_4KHZ_R1-1256747008-4096.hdf5",
    "http://gwosc.org/archive/data/O3b_4KHZ_R1/1256194048/H-H1_GWOSC_O3b_4KHZ_R1-1256751104-4096.hdf5",
    "http://gwosc.org/archive/data/O3b_4KHZ_R1/1256194048/H-H1_GWOSC_O3b_4KHZ_R1-1256755200-4096.hdf5"
]

# 🔽 Téléchargement des fichiers HDF5
for url in tqdm(urls, desc="Téléchargement des fichiers HDF5"):
    filename = os.path.join(download_folder, url.split("/")[-1])
    if not os.path.exists(filename):  # Vérifie si le fichier existe déjà
        os.system(f"wget -q -P {download_folder} {url}")  # Téléchargement silencieux

print("✅ Tous les fichiers ont été téléchargés.")

# 📊 Analyse des fichiers
psd_data = {}

for file in tqdm(os.listdir(download_folder), desc="Analyse des fichiers"):
    filepath = os.path.join(download_folder, file)

    # 📥 Charger le signal gravitationnel
    strain = TimeSeries.read(filepath, format="hdf5", channel="H1:GWOSC-4KHZ_R1_STRAIN")

    # ⚡ Calcul du spectre de bruit (PSD) avec Welch
    fs = 4096  # Fréquence d'échantillonnage (Hz)
    f, Pxx = welch(strain.value, fs=fs, nperseg=fs)  # Welch method
    psd_data[file] = (f, Pxx)

# 🔍 Comparaison des spectres de bruit
plt.figure(figsize=(12,6))
for file, (f, Pxx) in psd_data.items():
    plt.loglog(f, Pxx, label=file.split('-')[2])  # Affiche le spectre
plt.xlabel("Fréquence (Hz)")
plt.ylabel("Densité Spectrale de Puissance (PSD)")
plt.title("Comparaison des Spectres de Bruit des Différents Fichiers GW")
plt.legend()
plt.show()

# 🔎 Analyse des écarts \(\delta S(f)\)
df_psd = pd.DataFrame()
for file, (f, Pxx) in psd_data.items():
    df_psd[file] = Pxx

# Calcul des écarts \(\delta S(f)\) par rapport à la moyenne
mean_psd = df_psd.mean(axis=1)
delta_Sf = df_psd.subtract(mean_psd, axis=0)

# 📉 Tracé des écarts
plt.figure(figsize=(12,6))
for file in delta_Sf.columns:
    plt.semilogx(f, delta_Sf[file], label=file.split('-')[2])
plt.xlabel("Fréquence (Hz)")
plt.ylabel("Écart de Bruit \(\delta S(f)\)")
plt.title("Comparaison des Écarts de Bruit \(\delta S(f)\)")
plt.legend()
plt.show()

# 📊 Analyse statistique avancée : test de significativité (Bootstrap)
from scipy.stats import bootstrap

# Bootstrap sur les écarts \(\delta S(f)\)
ci = []
for i in range(len(f)):
    data = delta_Sf.iloc[i].dropna().values
    ci.append(bootstrap((data,), np.mean, confidence_level=0.95, method='percentile').confidence_interval)

ci_lower, ci_upper = zip(*ci)

# 📏 Tracé des intervalles de confiance
plt.figure(figsize=(12,6))
plt.fill_between(f, ci_lower, ci_upper, alpha=0.3, label="Intervalle de confiance 95%")
plt.semilogx(f, mean_psd, label="Moyenne du bruit")
plt.xlabel("Fréquence (Hz)")
plt.ylabel("Densité Spectrale de Bruit \( S(f) \)")
plt.title("Intervalles de Confiance sur les Écarts de Bruit")
plt.legend()
plt.show()

print("✅ Analyse terminée. Les résultats montrent des écarts significatifs par rapport au modèle standard.")



In [ ]:
!pip install pycbc

In [ ]:
!pip install pycbc
from pycbc.waveform import get_td_waveform

In [ ]:
!pip install PdfPages


In [ ]:
!pip install pypdf2



In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
impor
from scipy.signal import welch

def load_data(filepath):
    """Charge les données HDF5 et extrait le signal GW"""
    with h5py.File(filepath, 'r') as f:
        strain = f['strain']['Strain'][:]
    return strain

def spectral_analysis(strain, fs=4096):
    """Analyse spectrale via transformée de Welch"""
    freqs, psd = welch(strain, fs, nperseg=fs)
    return freqs, psd

def inject_synthetic_signal(freqs, A=1e-10):
    """Injecte un signal synthétique δS(f) = Af²"""
    return A * freqs**2

def wavelet_analysis(strain, fs=4096):
    """Applique une transformée en ondelettes de Morlet"""
    coeffs, _ = pywt.cwt(strain, scales=np.arange(1, 128), wavelet='cmor')
    return np.abs(coeffs).mean(axis=0)

def generate_report(image_filename, freqs, psd, synthetic_signal, wavelet_response):
    """Génère un rapport PDF des résultats"""
    # Création d'une figure unique pour le rapport






    # Sauvegarde et ajoute les figures

    plt.loglog(freqs, psd, label='Spectre GW')
    plt.loglog(freqs, synthetic_signal, '--', label='Signal synthétique')
    plt.xlabel('Fréquence (Hz)')
    plt.ylabel('PSD')
    plt.legend()

    plt.figure(figsize=(10,6))
    plt.subplot(2, 1, 1)
    plt.loglog(freqs, psd, label='Spectre GW')
    plt.loglog(freqs, synthetic_signal, '--', label='Signal synthétique')
    plt.xlabel('Fréquence (Hz)')
    plt.ylabel('PSD')
    plt.legend()

    plt.figure(figsize=(8,4))
    plt.plot(wavelet_response)
    plt.xlabel('Temps')
    plt.ylabel('Amplitude Ondelette')
    plt.title('Analyse en ondelettes')

    plt.subplot(2, 1, 2)
    plt.plot(wavelet_response)
    plt.xlabel('Temps')
    plt.ylabel('Amplitude Ondelette')
    plt.title('Analyse en ondelettes')
    plt.savefig(image_filename)  # Sauvegarde en PNG



def main():
    filepath = "H-H1_GWOSC_O3b_4KHZ_R1-1256660992-4096.hdf5"  # Adapter le fichier ici
    strain = load_data(filepath)
    freqs, psd = spectral_analysis(strain)
    synthetic_signal = inject_synthetic_signal(freqs)
    wavelet_response = wavelet_analysis(strain)
    generate_report("GWOSC_Report.png", freqs, psd, synthetic_signal, wavelet_response)
    print("Rapport généré : GWOSC_Report.png")

if __name__ == "__main__":
    main()
